# Looping Isochrones in Bits
This notebook uses the [NYC Vaccine Location dataset](https://vaccinefinder.nyc.gov/locations) and creates a 20-minute walkshed isochrone for each location. However, this dataset contains over 100 locations and can not be processed in one notebook without overloading the memory. Therefore, I'll run this notebook for 20 locations at a time (that appears to just max out the memory). For each chunk of locations, I'll only need to change one cell that subsets the raw dataset. At the end of the code, I'll save out the final dataframe as a .csv. Then, in a separate notebook, I'll bring in each dataframe and bind them together. 


## Initialize Workspace

First, I'll import libraries needed for analysis.

In [ ]:
# for data
import pandas as pd

# for spatial data
import geopandas as gpd

# for plotting
import matplotlib.pyplot as plt

# for network analysis
import networkx as nx

# for street network analysis
import osmnx as ox

# for basemaps
import contextily as ctx

Now, I'll upload the location dataset. We got this dataset by saving a geojson file from the ArcGIS server. The dataset is not live. It might be working re-checking in a couple weeks to see if any additional locations have been added. 

In [ ]:
# read in the geojson
raw = gpd.read_file('data/vac.geojson')

In [ ]:
# subset the dataset to process
# reset index so the loops work
# 1 = 0:20, 2 = 20:40, 3 = 40:60, 4 = 60:80, 5 = 80:100, 6 = 100:126
vac = raw[100:126].reset_index().copy()

# Set-Up for Isochrones

## Set-Up for Isochrones

### Download and Prep Street Network

For the final project, we will only look at a 15-minute walkshed. We're more interested in who has easy access (defined as a 15-minute walk), rather than gradations of access. 

In [ ]:
# set up variables
network_type = 'walk' # create walkshed
trip_times = 15 # minutes
meters_per_minute = 75 # travel distance per minute

The `place` variable will be a list of lat/lon tuples. This is created by initialize an empty list, the length of `vac` and then looping through each location to assign the lat/lon tuple to the list. 

In [ ]:
# create list of tuples for lat/lon

# iniatlize empty list
place = [0]*len(vac)

# loop through lat/lons
for i in range(len(vac)):
    place[i] = (vac.Latitude[i], vac.Longitude[i])

Now download the street network for each location. I will store the networks in a dictionary and name them G0, G1, G2, etc.

In [ ]:
%%time
# download multiple street networks into a dictionary using latlon list

# initialize empty dictionary
nets = {}

for i in range(len(vac)):
    name = "G" + str(i) # allows me to dynamically call each dataframe in the dictionary
    nets[name] = ox.graph_from_point(place[i], dist = 2000, network_type=network_type)

### Project Coordinates

Project each of the networks contained in the `nets` dictionary to Web mercator. This step also take some time.

In [ ]:
# Project network data to Web Mercator (measurements are in meters)

for i in range(len(vac)):
    name = "G" + str(i)
    nets[name] = ox.project_graph(nets[name], to_crs='epsg:3857')

### Convert Edges and Nodes to Geodataframes

Rather than creating a dataframe for nodes and edges, I will create separate dictionaries called `nodes` and `edges` that contain multiple dataframes (ex. gdf_nodes0, gdf_notes1...) 

In [ ]:
# convert nodes and edges to geodataframes stored in dictionaries

# initialize empty dictionaries
nodes = {}
edges = {}

# loop through
for i in range(len(vac)):
    node_name = "gdf_nodes" + str(i)
    edges_name = "gdf_edges" + str(i)
    nodes[node_name], edges[edges_name] = ox.graph_to_gdfs(nets['G' + str(i)]) 
    

### Get Centroids

Find the centroid for each location and store in dictionary.

In [ ]:
# initialize dictionaries
minx = {}
miny = {}
maxx = {}
maxy = {}

# loop through for the bounding box coordinates
for i in range(len(vac)):
    name = "gdf_nodes" + str(i)
    minx[name], miny[name], maxx[name], maxy[name] = nodes[name].geometry.total_bounds


In [ ]:
# initalize dictionaries
cenx = {}
ceny = {}

# loop through to calculate the centroid
for i in range(len(vac)):
    name = "gdf_nodes" + str(i)
    cenx[name] = (maxx[name]-minx[name])/2 + minx[name]
    ceny[name] = (maxy[name]-miny[name])/2 + miny[name]


### Get Nearest Node to Centroid

Apply the same dictionary loop methodology to get the center node for each location.

In [ ]:
cen_node = {}

# use osmnx's get_nearest_node command to get the id for the nearest node
for i in range(len(vac)):
    nets_name = 'G' + str(i)
    cen_name = "gdf_nodes" + str(i)
    
    cen_node[cen_name] = ox.get_nearest_node(nets[nets_name], 
                                  (ceny[cen_name],cenx[cen_name]), 
                                  method = 'euclidean')

## Create Isochrones 

### Calculate Travel Costs

In order to add a new column to each of the dataframes in the dictionary, I extract the dataframe `gdf_edgesi` from the `edges` dictionary, assign it to `dummy`, and then add the column. This is a way of directly editing the dataframe within the dictionary.

In [ ]:
# create a new column, calculate the time it takes to travel that edge
for i in range(len(vac)):
    name = 'gdf_edges' + str(i)
    dummy = edges[name]
    dummy['walk_time'] = dummy['length']/meters_per_minute

### Choose Colors

We'll only use one color, might as well make it a cool one.

In [ ]:
# assign time and color variables 
time = trip_times
color = '#228B22'

### Color Nodes Based on Travel Time

This step originally had two nested `for` loops. In this version, I wrap those initial nested `for` loops in an additional `for` loop that goes through each location.

In [ ]:
# loop through each trip time and associated color
subgraph = {}

for i in range(len(vac)):
    nets_name = "G" + str(i)
    nodes_name = "gdf_nodes" + str(i)
        
    subgraph[nets_name] = nx.ego_graph(nets[nets_name], cen_node[nodes_name], radius=time, distance='time')

    # for each of those nodes, update the gdf_nodes dataframe and assign it with its associated distance color
    for node in subgraph[nets_name].nodes():
        nodes[nodes_name].loc[node,'time'] = str(time) + ' mins'
        nodes[nodes_name].loc[node,'color'] = color

In [ ]:
# the NaN values then need to be populated with a valid color
for i in range(len(vac)): 
    name = "gdf_nodes" + str(i)
    dummy = nodes[name]
    dummy['color'].fillna('#cccccc', inplace=True)

### Dissolve and Create Convex Hulls

This step dissolves each dataframe in the `nodes` dictionary by time, resulting in four rows. Then a convex hull is created for each of the time categories.

In [ ]:
# dissolve the nodes by time
# collapses into a single row based on a certain column and results in multi-point geometry
# this will result in four rows, one for each time category

isochrones = {}

for i in range(len(vac)):
    nodes_name = "gdf_nodes" + str(i)
    dummy = nodes[nodes_name]
    isochrones[nodes_name] = dummy.dissolve("time")


In [ ]:
# for each row, create a convex hull

for i in range(len(vac)):
    nodes_name = "gdf_nodes" + str(i)
    dummy = isochrones[nodes_name]
    isochrones[nodes_name] = dummy.convex_hull.reset_index()

In [ ]:
# geometry header has been automatically named "0"
# let's rename that

for i in range(len(vac)):
    nodes_name = "gdf_nodes" + str(i)
    dummy = isochrones[nodes_name]
    dummy.columns=['time','geometry']

## Export

Currently, all the isochrones are in a dictionary. Let's put all the isochrones in a single dataframe with each isochrone as one row

In [ ]:
# collapse all rows into a single dataframe
iso6 = pd.concat(isochrones, axis=0).sum(axis=1, level=0)

In [ ]:
# export to .csv
iso6.to_csv('data/iso6.csv')